# GRU  + ElMO



In [1]:
import pandas as pd
import csv

df_dev = pd.read_table("../../../data/es/dev_es.tsv", index_col="id", quoting=csv.QUOTE_NONE)
df_train = pd.read_table("../../../data/es/train_es.tsv", index_col="id", quoting=csv.QUOTE_NONE)

text_train, y_train = df_train["text"], df_train["HS"]
text_dev, y_dev = df_dev["text"], df_dev["HS"]

print("Instancias de entrenamiento: {}".format(len(df_train)))
print("Instancias de desarrollo: {}".format(len(df_dev)))


Instancias de entrenamiento: 4500
Instancias de desarrollo: 500


Tengo que hacer dos cosas:

- Primero, convertir los tweets a secuencias de texto
- Luego, paddear las secuencias a cierta longitud (Keras necesita esto para poder paralelizar cálculo)

In [2]:
from nltk.tokenize import TweetTokenizer
from keras.preprocessing.sequence import pad_sequences


max_length = 30

tokenizer = TweetTokenizer(preserve_case=False, reduce_len=True, strip_handles=True)

def preprocess_tweet(tweet):
    tokens = tokenizer.tokenize(tweet)
    
    if len(tokens) >= max_length:
        tokens = tokens[:max_length]
    else:
        tokens = tokens + [''] * (max_length - len(tokens))
    return tokens


text_train = [preprocess_tweet(tweet) for tweet in df_train["text"].values]
text_dev = [preprocess_tweet(tweet) for tweet in df_dev["text"].values]

Using TensorFlow backend.


In [3]:
%%capture
from elmoformanylangs import Embedder

e = Embedder("../../../models/elmo/es/")

Carguemos embeddings

In [4]:
import numpy as np

print(text_train[0])


X_train = np.array(e.sents2elmo(text_train))
X_dev = np.array(e.sents2elmo(text_dev))

['easyjet', 'quiere', 'duplicar', 'el', 'número', 'de', 'mujeres', 'piloto', "'", 'verás', 'tú', 'para', 'aparcar', 'el', 'avión', '..', 'http://t.co/46NuLkm09x', '', '', '', '', '', '', '', '', '', '', '', '', '']


In [5]:
X_train.shape, X_dev.shape, y_train.shape, y_dev.shape

((4500, 30, 1024), (500, 30, 1024), (4500,), (500,))

In [6]:
from keras.models import Sequential
from keras.layers import Dense, Embedding, Dropout, LSTM
from keras.optimizers import Adam
from keras.preprocessing import sequence

embedding_dim = 1024

optimizer_args = {
    "lr": 0.0005,
    "decay": 0.01
}

model = Sequential()
model.add(LSTM(256, input_shape=(max_length, embedding_dim)))
model.add(Dropout(0.75))
model.add(Dense(256, activation='relu'))
model.add(Dropout(0.5))
model.add(Dense(1, activation='sigmoid'))
model.compile(loss='binary_crossentropy', 
              optimizer=Adam(**optimizer_args), 
              metrics=['accuracy'])
print(model.summary())

model.fit(X_train, y_train, validation_data=(X_dev, y_dev), epochs=20, batch_size=32)


_________________________________________________________________
Layer (type)                 Output Shape              Param #   
lstm_1 (LSTM)                (None, 256)               1311744   
_________________________________________________________________
dropout_1 (Dropout)          (None, 256)               0         
_________________________________________________________________
dense_1 (Dense)              (None, 256)               65792     
_________________________________________________________________
dropout_2 (Dropout)          (None, 256)               0         
_________________________________________________________________
dense_2 (Dense)              (None, 1)                 257       
Total params: 1,377,793
Trainable params: 1,377,793
Non-trainable params: 0
_________________________________________________________________
None
Train on 4500 samples, validate on 500 samples
Epoch 1/20
4500/4500 [==============================] - 9s 2ms/step - loss: 0.71

In [7]:
from hate.utils import print_evaluation

print_evaluation(model, X_dev, y_dev)

500/500 [==============================] - 0s 679us/step
Loss        : 0.4876
Accuracy    : 0.7740
Precision   : 0.7401
Recall      : 0.7568
F1          : 0.7483


## Bidirectional GRU

In [9]:
y_train.shape

(4500,)

In [15]:
from keras.models import Sequential
from keras.layers import Dense
from keras.layers import GRU, Dropout, LSTM, Bidirectional
from keras.layers.embeddings import Embedding
from keras.preprocessing import sequence
from keras.optimizers import Adam

optimizer_args = {
    "lr": 0.0005,
    "decay": 0.01
}

embedding_dim = 1024

model = Sequential()
model.add(Bidirectional(GRU(256, input_shape=(max_length, embedding_dim))))
model.add(Dropout(0.80))
model.add(Dense(256, activation='relu'))
model.add(Dropout(0.55))
model.add(Dense(1, activation='sigmoid'))
model.compile(loss='binary_crossentropy', 
              optimizer=Adam(**optimizer_args), 
              metrics=['accuracy'])

model.fit(X_train, y_train.values, validation_data=(X_dev, y_dev), epochs=25, batch_size=32)


Train on 4500 samples, validate on 500 samples
Epoch 1/25
4500/4500 [==============================] - 12s 3ms/step - loss: 0.8192 - acc: 0.5482 - val_loss: 0.6684 - val_acc: 0.5880
Epoch 2/25
4500/4500 [==============================] - 10s 2ms/step - loss: 0.7269 - acc: 0.5836 - val_loss: 0.6510 - val_acc: 0.6400
Epoch 3/25
4500/4500 [==============================] - 10s 2ms/step - loss: 0.6674 - acc: 0.6291 - val_loss: 0.6370 - val_acc: 0.6720
Epoch 4/25
4500/4500 [==============================] - 10s 2ms/step - loss: 0.6506 - acc: 0.6471 - val_loss: 0.6305 - val_acc: 0.6680
Epoch 5/25
4500/4500 [==============================] - 10s 2ms/step - loss: 0.6311 - acc: 0.6547 - val_loss: 0.6102 - val_acc: 0.7020
Epoch 6/25
4500/4500 [==============================] - 10s 2ms/step - loss: 0.6005 - acc: 0.6802 - val_loss: 0.5947 - val_acc: 0.7100
Epoch 7/25
4500/4500 [==============================] - 10s 2ms/step - loss: 0.5812 - acc: 0.6976 - val_loss: 0.5789 - val_acc: 0.7260
Epoch 8/

In [17]:
model.fit(X_train, y_train.values, validation_data=(X_dev, y_dev), epochs=2, batch_size=32)


Train on 4500 samples, validate on 500 samples
Epoch 1/2
4500/4500 [==============================] - 10s 2ms/step - loss: 0.3734 - acc: 0.8387 - val_loss: 0.4570 - val_acc: 0.7960
Epoch 2/2
4500/4500 [==============================] - 10s 2ms/step - loss: 0.3666 - acc: 0.8333 - val_loss: 0.4473 - val_acc: 0.7960


In [18]:
model.fit(X_train, y_train.values, validation_data=(X_dev, y_dev), epochs=3, batch_size=32)


Train on 4500 samples, validate on 500 samples
Epoch 1/3
4500/4500 [==============================] - 10s 2ms/step - loss: 0.3636 - acc: 0.8418 - val_loss: 0.4557 - val_acc: 0.7900
Epoch 2/3
4500/4500 [==============================] - 10s 2ms/step - loss: 0.3614 - acc: 0.8422 - val_loss: 0.4525 - val_acc: 0.7920
Epoch 3/3
4500/4500 [==============================] - 10s 2ms/step - loss: 0.3607 - acc: 0.8436 - val_loss: 0.4424 - val_acc: 0.8020


In [19]:
model.fit(X_train, y_train.values, validation_data=(X_dev, y_dev), epochs=3, batch_size=32)

Train on 4500 samples, validate on 500 samples
Epoch 1/3
4500/4500 [==============================] - 10s 2ms/step - loss: 0.3524 - acc: 0.8413 - val_loss: 0.4523 - val_acc: 0.7980
Epoch 2/3
4500/4500 [==============================] - 10s 2ms/step - loss: 0.3431 - acc: 0.8556 - val_loss: 0.4505 - val_acc: 0.7980
Epoch 3/3
4500/4500 [==============================] - 10s 2ms/step - loss: 0.3403 - acc: 0.8538 - val_loss: 0.4456 - val_acc: 0.8000


In [20]:
from hate.utils import print_evaluation

print_evaluation(model, X_dev, y_dev)

500/500 [==============================] - 1s 1ms/step
Loss        : 0.4456
Accuracy    : 0.8000
Precision   : 0.7675
Recall      : 0.7883
F1          : 0.7778


## Error Analysis

Vamos a ver los tweets con mayores errores

In [21]:
df_dev["proba"] = model.predict_proba(X_dev)


true_positives = df_dev[(df_dev["HS"] == 1) & (df_dev["proba"] >= 0.5)].copy()
true_negatives = df_dev[(df_dev["HS"] == 0) & (df_dev["proba"] < 0.5)].copy()

false_positives = df_dev[(df_dev["HS"] == 0) & (df_dev["proba"] > 0.5)].copy()
false_positives.sort_values("proba", ascending=False, inplace=True)


false_negatives = df_dev[(df_dev["HS"] == 1) & (df_dev["proba"] < 0.5)].copy()
false_negatives.sort_values("proba", ascending=True, inplace=True)

conf_matrix = pd.DataFrame([
    {"real":"hs=1", "pred_true": len(true_positives), "pred_false": len(false_negatives)},
    {"real":"hs=0", "pred_true": len(false_positives), "pred_false": len(true_negatives)}
])



conf_matrix.set_index("real", inplace=True)

conf_matrix[["pred_true", "pred_false"]]

pred_true  pred_false
real                       
hs=1        175          47
hs=0         53         225

In [27]:
false_negatives.iloc[:20]


text  \
id                                                                                                                                                                                                               
21535                           #VOX critica duramente a ⁦@pablocasado_⁩ “Les da la bienvenida como si fuera un cartel del welcome de #Carmena “ ¡Expulsión inmediata de moromierdas!  https://t.co/HaySDibj2l   
21152  @NatyBurgos15 @Horaciogenta @PaolaPa05 Cállate vos Black and bitch como la puta falsa abogada. Espero se pudra en la cárcel y devuelva todo. Dedícate a estudiar en serio. La droga es mala. Cristin...   
22560                                                                                                             @CarlaEUrena @FelipeCalderon Callateeeee !! Puta gallina cacarea !! Ni idea tienes de nada .   
23415                                                                                                                                      Exacto! #puralata #feminazis #HombreAbusado https://t.co/zTEvaftxcd   
21215                    Que tal otro aporte anónimo son de la puta de mi esposa, así los dejó en navidad enseñando le los calzones al vecino rt y siguenos @TANGASUCIA @Tangasrobadas https://t.co/70ioUjveqY   
24925                                                         La verdad que si, no veia la hora de llegar a los 5 mil amigos😂 ahora de a poco voy borrando esos indios indocumentados😂 https://t.co/Xw8q4GLM8v   
24401  No habrá que darles de lo 'nuestro'. Ellos se apropiarán de lo 'suyo'.  @Manuj_Hidalgo les dice ☝️ a quienes piensan q hay q jubilar a los mayores para q trabajen los jóvenes o q los q vienen d fu...   
21072  Voy andando por la calle y me han dicho cosas en árabe ya que me estas acosando dime lo en español para que te entienda, asique como yo no les entiendo me he cagado en su puta madre en árabe para ...   
24590                                     SOMOS MUCHO MÁS DE LO QUE SUEÑAS SER PINCHÉ SUDACA CAGADO, y si no te gusta dime donde nos topamos pa reventarnos el hocico 1:1 @amarchesin1 https://t.co/vHHPBKS1AK   
22008                                                                                        @Namurii @medicaster_xz @Yorha2d @KeloKlau_ Se merecía de puta y no quiere que le metan la verga, jajaja no mamen   
24488                                                                                        Un indocumentado...habla y habla y no para y acaba hablando su ignorancia.Empezamos bien. https://t.co/SMRsqjtEBT   
22168                                                                               Mi novia es como mi polla. Sé que está ahí porque de vez en cuando se levanta. Una de mis pantalones y la otra de la cama.   
24864                                                                                                                                                    rosalia me podría decir sudaca i wouldn't give a fuck   
24481                                                                                                           En que se parece superman a una mujer lista: En que los dos son personajes de ciencia-ficción.   
24431                                                              Yo nunca le he dicho puta a una mujer pero socia tú eres senda petardo 😂😂 estas navidades no voy a comprar pirotecnias ya contigo vasta 😂😂😂   
24746                                                                                    @rU_1970 muestra como teni esa vagina de puta caliente llena de moco wacha culia hija de la perra traga semen 146:146   
21887  @A3Noticias Esta tiene que ser una 'fake new' porque las mujeres son siempre las víctimas, no las hijas de puta maltratadoras. Eso o el chaval es un machista que se lo ha merecido (machete al mach...   
21727  España crea su primer 'campo de refugiados' en la bahía de Algeciras https://t.co/l7Zmh2kvPN vía @elmundoes Gran noticia para el contribuyente español. Españoles a pagar, a pagar. Las pensiones pe...   
20

- Agregar los hashtag! Estamos perdiendo información con eso
- También las masyúsculas...


In [28]:
pd.set_option('max_colwidth', 200)
false_positives.iloc[:20]

text  \
id                                                                                                                                                                                                               
22517                                                     -¿eres poblana? -soy colombiana -¿y por qué eres güera? en Colombia TODAS son morenas, fui a Colombia 3 veces. -ok, mentí 🙄 solo me pinto el cabello   
22416                                         @Pepe_ElEzequiel Puta!? Puta con las que vos te metes Esa mujer es un ángel enviado por Jesucristo nuestro señor para darnos esa paz que buscamos sólo con verla   
21892                                                                                                                                                              @saancisco Callate puta tu eres el traidor.   
21497                                                                                                                                                        Talvez quede como zorra pero te lo mereces Socia😎   
24172                   Y los que no se denuncian... hagamos números. El acoso a las mujeres es diario. No todos los hombres son acosadores, pero todas las mujeres han sido acosadas. https://t.co/jza9llhqWA   
21255                                                               Y tú eres una maldita escoria con una repugnante inclinación por los hombres.. ¡Seres como tú me producen náuseas! https://t.co/TaHDcpZdnm   
24159                                                                                         @LeonDeJapon Jajajajajaja tu eres uno, diego jajajajajaja mentira perra perra perrraaaaaaaaaa de mierda jajaja ♡   
24402                                                                                                                            zorra eres tu que no te sientes hombre y no entiendes que toda mujer es bella   
21671  @danlntifada @fatinascidi @manolivizoso @Zurine3 @Jvillaver2 https://t.co/HG4ljNlwR3, coño entonces tu eres de lo que te guias por el antes de cristo y despues de cristo,cuando llegamos los Españo...   
22115                                                                                                                     Una chica no es una guarra, una zorra, una puta, una buscan por su manera de vestir.   
22014                                                                                                                                        Ni tú eres una zorra hambrienta ni yo un incorregible depredador.   
20376                                                                                                                                                                                Zorra tu abuela, imbécil.   
21050                                                                                                                                                 @GvyMC @Wismichu Callate puta rata😂😂 ya estas llorando??   
22848                                                                                                                    Pinche bebé pendejo cállate el hocico, nomás te están cortando el pelo. Puta chillona   
22938                                                                                      Masculinidad frágil es mi crush francés echándose un clavado al cenote solo porque un grupo de árabes le dijo pussy   
23320                                                                 @HenriFalconLara No digas nada que tú eres cómplice de lo que está pasando... y recuerda seguirás siendo la perra más perra de Venezuela   
21966                          escúchame ya me jodería jajajajajajajajajajajajajaja si pa ti enseñar es ser una guarra yo soy biennnn guarra, menos mal que no me respeto que si no... https://t.co/yt5fHp99b7   
24209                                                                ¿Ya les he contado que si fuera rica/millonaria sería toda una PERRA?, Yisus me está enseñando humildad... Y no he aprendido mucho ldvdd.   
22